In [73]:
import numpy as np

def conjugate_gradient(A, b, tol=1e-8, max_iter=1000):
    """
    Conjugate Gradient Method for solving Ax = b.
    """
    x = np.zeros(b.shape)
    r = b - np.dot(A, x)
    p = r.copy()
    rs_old = np.dot(r, r)
    
    for i in range(max_iter):
        Ap = np.dot(A, p)
        alpha = rs_old / np.dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        rs_new = np.dot(r, r)
        if np.sqrt(rs_new) < tol:
            break
        beta = rs_new / rs_old
        p = r + beta * p
        rs_old = rs_new
    
    return x, i+1

def jacobi_preconditioner(A):
    """
    Creates a Jacobi preconditioner for matrix A.
    """
    M = np.diag(1 / np.diag(A))
    return M

def ssor_preconditioner(A, omega=1.0):
    """
    Creates an SSOR preconditioner for matrix A with relaxation factor omega.
    """
    D = np.diag(np.diag(A))
    L = np.tril(A, -1)
    U = np.triu(A, 1)
    M_inv = np.linalg.inv(D + omega * L) @ D @ np.linalg.inv(D + omega * U)
    return M_inv

def preconditioned_conjugate_gradient(A, b, M_inv, tol=1e-8, max_iter=1000):
    """
    Preconditioned Conjugate Gradient Method for solving Ax = b.
    """
    x = np.zeros(b.shape)
    r = b - np.dot(A, x)
    z = np.dot(M_inv, r)
    p = z.copy()
    rs_old = np.dot(r, z)
    
    for i in range(max_iter):
        Ap = np.dot(A, p)
        alpha = rs_old / np.dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        z = M_inv @ r
        rs_new = np.dot(r, z)
        if np.sqrt(rs_new) < tol:
            break
        beta = rs_new / rs_old
        p = z + beta * p
        rs_old = rs_new
    
    return x, i+1

In [74]:
def create_problem_matrix(n):
    """
    Create the matrix A and vector b for the problem.
    """
    h = 1 / (n + 1)
    size = n * n
    A = np.zeros((size, size))
    b = h**2 * np.ones(size)
    
    for i in range(n):
        for j in range(n):
            idx = i * n + j
            A[idx, idx] = 4 + h**2
            if i > 0:  # -I on the subdiagonal
                A[idx, idx - n] = -1
            if i < n - 1:  #  -I on the superdiagonal
                A[idx, idx + n] = -1
            if j > 0:  # -1 on the subdiagonal of D
                A[idx, idx - 1] = -1
            if j < n - 1:  # -1 on the superiagonal of D
                A[idx, idx + 1] = -1
    
    return A, b


In [75]:
def compare_methods(n):
    """
    Compare the performance of CG, Jacobi-PCG, and SSOR-PCG for the given n.
    """
    A, b = create_problem_matrix(n)
    
    _, cg_iters = conjugate_gradient(A, b)
    
    M_inv_jacobi = jacobi_preconditioner(A)
    _, pcg_jacobi_iters = preconditioned_conjugate_gradient(A, b, M_inv_jacobi)
    
    M_inv_ssor = ssor_preconditioner(A)
    _, pcg_ssor_iters = preconditioned_conjugate_gradient(A, b, M_inv_ssor)
    
    return f'CG:{cg_iters}', f'Jacobi:{pcg_jacobi_iters}', f'SSOR:{pcg_ssor_iters}'

In [76]:
results = {}
for n in [8, 16, 32]:
    results[n] = compare_methods(n)
print(results)

{8: ('CG:10', 'Jacobi:10', 'SSOR:10'), 16: ('CG:26', 'Jacobi:26', 'SSOR:16'), 32: ('CG:53', 'Jacobi:52', 'SSOR:28')}
